In [36]:
import tensorflow as tf
import keypress
import numpy as np
import pandas as pd
import time
import csv
from pylsl import StreamInlet, resolve_stream
import statistics

In [37]:
dataset = pd.read_csv("MI_data.csv")
X = dataset.iloc[:, :8].values
y = dataset.iloc[:, -1].values


In [38]:
print(y)
print(y[400])
print(y[2500])
print(len(y))

['RL' 'RL' 'RL' ... 'LL' 'LL' 'LL']
RL
LL
2799


In [39]:
from sklearn.preprocessing import OneHotEncoder,LabelEncoder
label = LabelEncoder()
y = label.fit_transform(y)
y =y.reshape(-1,1)
ohe = OneHotEncoder()
y= ohe.fit_transform(y).toarray()

from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)


from sklearn.preprocessing import StandardScaler
sctr = StandardScaler()
X_train = sctr.fit_transform(X_train)
scte = StandardScaler()
X_test = scte.fit_transform(X_test)

In [40]:
print(y[400],y[2500])

[0. 1.] [1. 0.]


In [41]:
classifier = tf.keras.models.Sequential()

# Adding the input layer and the first hidden layer
classifier.add(tf.keras.layers.Dense(units=10, activation='relu'))

# Adding the second hidden layer
classifier.add(tf.keras.layers.Dense(units=10, activation='relu'))


# Adding the output layer
classifier.add(tf.keras.layers.Dense(units=2, activation='softmax'))

# Part 3 - Training the ANN

# Compiling the ANN
classifier.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# Training the ANN on the Training set
classifier.fit(X_train, y_train, batch_size=50, epochs=25)
y_pred = classifier.predict(X_test)

Epoch 1/25
45/45 [==============================] - 0s 793us/step - loss: 1.0730 - accuracy: 0.3193
Epoch 2/25
45/45 [==============================] - 0s 884us/step - loss: 0.6746 - accuracy: 0.3444
Epoch 3/25
45/45 [==============================] - 0s 748us/step - loss: 0.4768 - accuracy: 0.8723
Epoch 4/25
45/45 [==============================] - 0s 907us/step - loss: 0.3518 - accuracy: 0.9830
Epoch 5/25
45/45 [==============================] - 0s 771us/step - loss: 0.2626 - accuracy: 0.9830
Epoch 6/25
45/45 [==============================] - 0s 750us/step - loss: 0.1961 - accuracy: 0.9830
Epoch 7/25
45/45 [==============================] - 0s 793us/step - loss: 0.1455 - accuracy: 0.9830
Epoch 8/25
45/45 [==============================] - 0s 862us/step - loss: 0.1080 - accuracy: 0.9830
Epoch 9/25
45/45 [==============================] - 0s 771us/step - loss: 0.0800 - accuracy: 0.9830
Epoch 10/25
45/45 [==============================] - 0s 771us/step - loss: 0.0594 - accuracy: 0.9830

In [45]:
print("starting...")
streams = resolve_stream('type', 'EEG')
inlet = StreamInlet(streams[0])
duration = 0.5


starting...


In [47]:
def predictit():
    def sample_extract():
        start = time.time()
        with open('openbci.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            while time.time() <= start + duration:
                samples, timestamp = inlet.pull_chunk()
                if samples:
                    writer.writerows(samples)

    def processing_data():
        with open('prediction_data.csv', 'w', newline='') as csvfile:
            writer = csv.writer(csvfile, delimiter=',')
            for sam in range(0, 21):  # increase the range to get more recordings
                sample_extract()
                if sam == 0:
                    continue
                raw_dataset = pd.read_csv("openbci.csv")
                raw = raw_dataset.iloc[:, :].values
                print(len(raw))
                p = []
                for r in range(len(raw[0])):
                    p.append(np.average(raw[:, r]))
                print(p)
                writer.writerow([p[0], p[1], p[2], p[3], p[4], p[5], p[6], p[7]])

    processing_data()

    test = pd.read_csv("prediction_data.csv")
    lol = test.iloc[:, :].values
    sctt = StandardScaler()
    lol = sctt.fit_transform(lol)
    prediction = classifier.predict(lol)
    #time.sleep(15)
    final = []
    for d in range(len(prediction)):
        final.append(np.argmax(prediction[d]))
    ree = statistics.mode(final)
    time.sleep(1.5)
    if ree == 0:
        keypress.press('d')
    elif ree == 1:
        keypress.press('a')
    elif ree == 2:
        keypress.press('w')
    elif ree == 3:
        keypress.press('s')
    time.sleep(1.5)


while 1:
    predictit()

26623
[-5607.003174149085, -5702.084394727128, -5873.510953929491, -5888.722404067031, -112510.94685633662, -137594.77340140575, -83173.84039235342, -109382.31687298107]
26681
[-5599.148486582505, -5692.709158121943, -5865.265251134791, -5879.664833271013, -110427.06185200845, -135592.46150291874, -81697.14611960524, -107083.60124128593]
139
[-5593.792248622976, -5689.337163472347, -5860.400661111736, -5876.064909790917, -109518.99066996403, -134634.11207284173, -81069.85903776978, -106043.69222122303]
129
[-5592.62789183624, -5688.176924358043, -5858.674467811289, -5874.595161851987, -109509.10331879844, -134623.66690891472, -81062.7972989341, -106032.48504118217]
129
[-5590.650882691376, -5686.604840419089, -5856.639088238857, -5872.650212724079, -109498.54493701551, -134612.67490310076, -81054.9000121124, -106020.90958091085]
129
[-5593.635450732801, -5689.548354984254, -5859.795531280281, -5875.528384659641, -109492.43889292635, -134606.0909641473, -81051.43380571705, -106013.44452

KeyboardInterrupt: 